In [24]:
import cohere
import os
import time
import re
import pickle
import numpy as np
import json

In [5]:
# Ścieżka do folderu z plikami
dataset_path = "../../ML_Model/datasets/tested_dataset/final_dataset/final_test.pkl"

# Preprocessing dataset

In [36]:
# Wczytanie pliku
with open(dataset_path, 'rb') as f:
    data = pickle.load(f)

In [37]:
labels = data['labels']
answers = data['answer']
evaluations = data['rating']
fps = data['fps']

# Api

In [39]:
# Inicjalizacja klienta Cohere
# https://cohere.com/
# https://docs.cohere.com/reference/embed

with open('../api.txt', 'r', encoding='utf-8') as file:
    for line in file:
        if 'cohere:' in line:
            # Wyciągamy tekst po "cohere:"
            key_api = line.split('cohere:')[1].strip()
            
co = cohere.Client(key_api)

# Generowanie odpowiedzi

In [ ]:
# Funkcja do generowania analizy techniki z komentarzy
def generate_technique_analysis_prompt(data, prompt):
    prompt += "\n".join(data)  # Łączymy wszystkie komentarze w jeden ciąg
    
    response = co.generate(
        model='command',  # Poprawiony model
        prompt=prompt,
        max_tokens=300,
        temperature=0.7
    )
    
    # Zwracamy wygenerowaną odpowiedź
    return response.generations[0].text

In [ ]:
# Zbudowanie promptu na podstawie komentarzy
prompt = "Please analyze the following comments and provide a detailed analysis of exercise technique improvement: Get information from all comments below. Let your answer be coherent\n\n"

In [ ]:
analysis_comments_prompt = {}

# Wyświetlenie wyników
for i, data in enumerate(labels):
    start_time = time.time()

    print(f"Wyniki dla nagrania: {filename}, numer {i}")
    if i >= 3:
        break
        
    label = labels[i]
    answer = answers[i]
    evaluation = json.loads(evaluations[i])[0]["number"]
    fps_ = fps[i]

    data_input = [label, answer, evaluation, fps_]
    
    # Generowanie analizy techniki, jeśli komentarze istnieją
    if comments:
        analysis = generate_technique_analysis_prompt(data_input, prompt)
    else:
        analysis = "No comments available for analysis."
    
    # Dodajemy wynik do listy
    analysis_comments_prompt[filename] = {
        'Title': title,
        'URL': url,
        'Selftext': selftext,
        'Media': media,
        'Comments' : comments,
        'Analysis': analysis
    }
    
    print('*'*100)
    print(f"Title: {title}")
    print(f"URL: {url}")
    print(f"Selftext: {selftext}")
    print(f"Media: {media}")
    print('+'*100)
    print(f"Komentarze: \n {comments}\n")
    print('+'*100)
    print('-'*100)
    print(f"Analysis: \n {analysis}\n")
    print('-'*100)
    print('*'*100)
    
    end_time = time.time()
    print(f"Przetwarzanie trwało {end_time - start_time:.4f} sekundy")
    
    print("-" * 40)